In [ ]:
pip install transformers

In [ ]:
# main.py
# This script demonstrates how to fine-tune a pre-trained language model
# to become a specialized career suggestion assistant.

# --- 1. Installation ---
# You need to install the 'transformers' and 'torch' libraries first.
# Run these commands in your terminal:
# pip install transformers torch datasets

import os
import json
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

# --- 2. Create a Sample Dataset ---
# To train your model, you need a dataset of examples. Each example is a
# text string that shows the model what a good input and output look like.
# The format we'll use is a simple text file where each line is a JSON object.

# Let's create a sample dataset file named 'career_data.jsonl'
dataset_content = [
    {"text": "PROFILE: Name: John Anderson, Department: Marketing, Passing Year: 2022, Skills: SEO, Content Writing. SUGGESTION: John should focus on a 'Digital Marketing Specialist' role. He can enhance his profile by completing a Google Analytics certification and building a portfolio of successful SEO campaigns."},
    {"text": "PROFILE: Name: Priya Singh, Department: Computer Science, Passing Year: 2024, Skills: Python, Java, SQL. SUGGESTION: Priya is well-suited for a 'Backend Developer' role. To stand out, she should contribute to an open-source project on GitHub and create a personal project using a modern web framework like Django or FastAPI."},
    {"text": "PROFILE: Name: David Chen, Department: Business Administration, Passing Year: 2023, Skills: Financial Modeling, Excel, Public Speaking. SUGGESTION: David's skills align perfectly with a 'Financial Analyst' position. He should practice case studies from top investment firms and get certified in financial modeling (e.g., FMVA)."},
    {"text": "PROFILE: Name: Maria Garcia, Department: Graphic Design, Passing Year: 2021, Skills: Adobe Illustrator, Photoshop, UI/UX Principles. SUGGESTION: Maria should target 'UI/UX Designer' roles. Her next step should be to learn Figma and create a comprehensive portfolio on Behance showcasing 3-5 detailed app or web design projects."},
]

dataset_filename = "career_data.jsonl"
with open(dataset_filename, 'w') as f:
    for entry in dataset_content:
        f.write(json.dumps(entry) + '\n')

print(f"✅ Sample dataset created at '{dataset_filename}'")

# --- 3. Configure the Model ---
# We'll use a powerful but relatively small model from Hugging Face.
# 'distilgpt2' is a great starting point.
MODEL_NAME = "distilgpt2"
OUTPUT_DIR = "./career-ai-model" # Where the trained model will be saved

# Load the tokenizer, which prepares the text for the model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Add a padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load the pre-trained model
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
# Resize model embeddings to account for the new pad token
model.resize_token_embeddings(len(tokenizer))


# --- 4. Prepare the Dataset for Training ---
# We load the text file into a format the Trainer can use.
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128  # The size of text chunks to feed the model
    )

train_dataset = load_dataset(dataset_filename, tokenizer)

# The data collator groups the data into batches for training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False, # Masked Language Modeling is false for GPT-style models
)

print("✅ Dataset loaded and prepared for training.")

# --- 5. Set Up the Trainer ---
# The Trainer handles the entire training loop for us.
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,          # Directory to save the model
    overwrite_output_dir=True,      # Overwrite the content of the output directory
    num_train_epochs=10,            # Number of times to train on the dataset
    per_device_train_batch_size=2,  # Batch size for training
    save_steps=50,                  # How often to save a checkpoint
    save_total_limit=2,             # Only keep the last 2 checkpoints
    logging_steps=10,               # How often to log training progress
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# --- 6. Start Training ---
print("\n--- Starting Model Fine-Tuning ---")
trainer.train()
print("--- Model Fine-Tuning Complete ---\n")

# Save the final model and tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Final model saved to '{OUTPUT_DIR}'")


# --- 7. Use Your Trained Model for Suggestions ---
def generate_suggestion(profile_text):
    """
    Generates a career suggestion using your newly fine-tuned model.
    """
    # Load your fine-tuned model from the output directory
    loaded_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    loaded_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)

    # Prepare the input text
    prompt = f"PROFILE: {profile_text}. SUGGESTION:"
    inputs = loaded_tokenizer(prompt, return_tensors="pt")

    # Generate the output
    outputs = loaded_model.generate(
        inputs.input_ids,
        max_length=150,
        num_return_sequences=1,
        pad_token_id=loaded_tokenizer.pad_token_id,
        eos_token_id=loaded_tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50
    )

    # Decode and print the result
    generated_text = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the suggestion part
    suggestion = generated_text.split("SUGGESTION:")[1].strip()
    return suggestion

# --- Example Usage ---
print("\n--- Testing the Fine-Tuned Model ---")
new_profile = "Name: Samira Khan, Department: Electrical Engineering, Passing Year: 2023, Skills: C++, MATLAB, Circuit Design"
suggestion = generate_suggestion(new_profile)

print(f"Profile:\n{new_profile}\n")
print(f"Generated Suggestion:\n{suggestion}")

✅ Sample dataset created at 'career_data.jsonl'


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gnanesh847 (gnanesh847-sri-sivani-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.877600


--- Model Fine-Tuning Complete ---

✅ Final model saved to './career-ai-model'

--- Testing the Fine-Tuned Model ---
Profile:
Name: Samira Khan, Department: Electrical Engineering, Passing Year: 2023, Skills: C++, MATLAB, Circuit Design

Generated Suggestion:
Samira can be a perfect candidate for a position in Electrical Engineering, but she is just a special case. Her past job of computer science and math has helped her develop an understanding of the world of computing. She has done a lot on the computer science field, so this position is no exception. She can be a successful candidate for a position in Computer Science, but she is just a special case. Her past job of computer science and math has helped her develop an understanding of the world of computing. She has done a lot on the computer science field, so


Roadmap Generator **module**

In [ ]:
# main.py
# This script fine-tunes a model to generate multi-level career roadmaps.

# --- 1. Installation ---
# pip install transformers torch datasets

import os
import json
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

# --- 2. Create a Sample Dataset for Roadmaps ---
# This dataset is more structured. It teaches the model to generate
# a full roadmap based on a profile and an experience level.
roadmap_dataset_content = [
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Beginner. ROADMAP: ### Beginner Roadmap for Backend Developer\n\n**Month 1: Core Concepts**\n- **Courses:** Complete 'Introduction to Web Development' on Coursera. Study HTTP protocols and REST API principles.\n- **Project:** Build a simple command-line application that interacts with a public API (e.g., a weather app).\n\n**Month 2: Framework Fundamentals**\n- **Courses:** Learn the basics of a web framework like Django or FastAPI. Follow the official tutorials.\n- **Project:** Convert your command-line app into a basic web server with a few API endpoints.\n\n**Month 3: Database Essentials**\n- **Courses:** Take a course on SQL fundamentals and database design. Learn about ORMs (Object-Relational Mapping).\n- **Project:** Integrate a SQLite database into your web application to store and retrieve data."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Intermediate. ROADMAP: ### Intermediate Roadmap for Backend Developer\n\n**Month 1: Authentication & Security**\n- **Courses:** Study token-based authentication (JWT) and OAuth2. Learn about common security vulnerabilities (e.g., SQL injection, XSS).\n- **Project:** Implement user registration and login with secure password hashing and JWTs in your application.\n\n**Month 2: Docker & Containers**\n- **Courses:** Learn Docker fundamentals. Understand how to create Dockerfiles and manage containers with Docker Compose.\n- **Project:** Containerize your web application and its database into separate Docker containers.\n\n**Month 3: Testing & CI/CD**\n- **Courses:** Learn about unit testing and integration testing. Study the basics of CI/CD pipelines using GitHub Actions.\n- **Project:** Write unit tests for your API endpoints. Create a simple CI/CD pipeline that automatically runs tests when you push code to GitHub."},
    {"text": "PROFILE: David Chen, Business Administration, 2023, Financial Modeling, Excel. LEVEL: Beginner. ROADMAP: ### Beginner Roadmap for Financial Analyst\n\n**Month 1: Advanced Excel**\n- **Courses:** Master advanced Excel functions (VLOOKUP, INDEX-MATCH, PivotTables). Complete a course on financial modeling in Excel.\n- **Project:** Build a 3-statement financial model (Income Statement, Balance Sheet, Cash Flow) for a public company.\n\n**Month 2: Introduction to Python for Finance**\n- **Courses:** Learn basic Python syntax and data structures. Study the pandas library for data manipulation.\n- **Project:** Use Python and pandas to download stock price data and perform a simple moving average analysis.\n\n**Month 3: Valuation Techniques**\n- **Courses:** Study Discounted Cash Flow (DCF) and Comparable Company Analysis (CCA).\n- **Project:** Perform a DCF valuation for a company using the financial model you built in Month 1."},
]

dataset_filename = "roadmap_data.jsonl"
with open(dataset_filename, 'w') as f:
    for entry in roadmap_dataset_content:
        f.write(json.dumps(entry) + '\n')

print(f"✅ Roadmap dataset created at '{dataset_filename}'")

# --- 3. Configure the Model ---
MODEL_NAME = "distilgpt2"
OUTPUT_DIR = "./career-ai-roadmap-model"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

# --- 4. Prepare the Dataset for Training ---
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=256  # Increased block size for longer roadmap text
    )

train_dataset = load_dataset(dataset_filename, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
print("✅ Dataset loaded and prepared for training.")

# --- 5. Set Up the Trainer ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=15, # Increased epochs for better learning on a small dataset
    per_device_train_batch_size=1, # Smaller batch size for longer text
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# --- 6. Start Training ---
print("\n--- Starting Model Fine-Tuning for Roadmap Generation ---")
trainer.train()
print("--- Model Fine-Tuning Complete ---\n")

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ Final roadmap model saved to '{OUTPUT_DIR}'")

# --- 7. Use Your Trained Model for Roadmap Generation ---
def generate_roadmap(profile_text, experience_level):
    """
    Generates a career roadmap using the fine-tuned model.
    """
    # Load your fine-tuned model
    loaded_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    loaded_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)

    # Prepare the input prompt
    prompt = f"PROFILE: {profile_text}. LEVEL: {experience_level}. ROADMAP:"
    inputs = loaded_tokenizer(prompt, return_tensors="pt")

    # Generate the output
    outputs = loaded_model.generate(
        inputs.input_ids,
        max_length=300, # Longer max length for detailed roadmaps
        num_return_sequences=1,
        pad_token_id=loaded_tokenizer.pad_token_id,
        eos_token_id=loaded_tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_k=50
    )

    # Decode and extract the roadmap
    generated_text = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    roadmap = generated_text.split("ROADMAP:")[1].strip()
    return roadmap

# --- Example Usage ---
print("\n--- Testing the Fine-Tuned Roadmap Generator ---")
new_profile = "Samira Khan, Electrical Engineering, 2023, C++, MATLAB, Circuit Design"

print(f"Profile: {new_profile}\n")

print("--- Generating Beginner Roadmap ---")
beginner_roadmap = generate_roadmap(new_profile, "Beginner")
print(beginner_roadmap)

print("\n--- Generating Intermediate Roadmap ---")
intermediate_roadmap = generate_roadmap(new_profile, "Intermediate")
print(intermediate_roadmap)


✅ Roadmap dataset created at 'roadmap_data.jsonl'


/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


✅ Dataset loaded and prepared for training.

--- Starting Model Fine-Tuning for Roadmap Generation ---


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.911600
20,1.956600
30,1.557700


--- Model Fine-Tuning Complete ---

✅ Final roadmap model saved to './career-ai-roadmap-model'

--- Testing the Fine-Tuned Roadmap Generator ---
Profile: Samira Khan, Electrical Engineering, 2023, C++, MATLAB, Circuit Design

--- Generating Beginner Roadmap ---
Model the basic layout of a web application. VIEWS: Basic. VIEWS: Core concepts and concepts for creating and deploying a web application using Python, SQLite, and more. METHODS: Model a web application using Django. VIEWS: Learn the basics of a web application or application that stores user data and sends it to the web server. VIEWS: Learn about the basics of a web application or application that stores user data and sends it to the web server. VIEWS: Learn about the basics of a web application or application that stores user data and send it to the web server. VIEWS: Learn about the basics of a web application or application that stores user data and sends it to the web server. VIEWS: Learn about the basics of a web applicati

In [ ]:
pip install gtts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [ ]:
# main.py
# This script fine-tunes a model and then uses a powerful LLM to generate
# detailed course content with voice from a user's roadmap.

# --- 1. Installation ---
# pip install transformers torch datasets requests gtts
# 'requests' is used to interface with OpenRouter, and 'gtts' for text-to-speech.

import os
import json
import requests # Using the requests library to connect to OpenRouter
from gtts import gTTS
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

# --- 2. Create an Expanded Sample Dataset for Roadmaps ---
# Added an "Advanced" level to make the fine-tuned model more robust.
roadmap_dataset_content = [
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Beginner. ROADMAP: ### Beginner Roadmap for Backend Developer\n\n**Month 1: Core Concepts**\n- **Courses:** Complete 'Introduction to Web Development' on Coursera. Study HTTP protocols and REST API principles.\n- **Project:** Build a simple command-line application that interacts with a public API (e.g., a weather app).\n\n**Month 2: Framework Fundamentals**\n- **Courses:** Learn the basics of a web framework like Django or FastAPI. Follow the official tutorials.\n- **Project:** Convert your command-line app into a basic web server with a few API endpoints.\n\n**Month 3: Database Essentials**\n- **Courses:** Take a course on SQL fundamentals and database design. Learn about ORMs (Object-Relational Mapping).\n- **Project:** Integrate a SQLite database into your web application to store and retrieve data."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Intermediate. ROADMAP: ### Intermediate Roadmap for Backend Developer\n\n**Month 1: Authentication & Security**\n- **Courses:** Study token-based authentication (JWT) and OAuth2. Learn about common security vulnerabilities (e.g., SQL injection, XSS).\n- **Project:** Implement user registration and login with secure password hashing and JWTs in your application.\n\n**Month 2: Docker & Containers**\n- **Courses:** Learn Docker fundamentals. Understand how to create Dockerfiles and manage containers with Docker Compose.\n- **Project:** Containerize your web application and its database into separate Docker containers.\n\n**Month 3: Testing & CI/CD**\n- **Courses:** Learn about unit testing and integration testing. Study the basics of CI/CD pipelines using GitHub Actions.\n- **Project:** Write unit tests for your API endpoints. Create a simple CI/CD pipeline that automatically runs tests when you push code to GitHub."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Advanced. ROADMAP: ### Advanced Roadmap for Backend Developer\n\n**Month 1: System Design & Scalability**\n- **Courses:** Study system design principles like load balancing, caching, and message queues (e.g., RabbitMQ, Kafka).\n- **Project:** Design a scalable architecture for a social media application, focusing on handling high traffic.\n\n**Month 2: Microservices & Orchestration**\n- **Courses:** Learn about microservice architecture. Study Kubernetes for container orchestration.\n- **Project:** Break down your monolithic application into several microservices and deploy them on a local Kubernetes cluster (Minikube).\n\n**Month 3: Performance Monitoring & Optimization**\n- **Courses:** Learn to use monitoring tools like Prometheus and Grafana. Study database indexing and query optimization.\n- **Project:** Set up a monitoring dashboard for your microservices. Identify and fix a performance bottleneck in your application."},
]

dataset_filename = "roadmap_data.jsonl"
with open(dataset_filename, 'w') as f:
    for entry in roadmap_dataset_content:
        f.write(json.dumps(entry) + '\n')

print(f"✅ Roadmap dataset created at '{dataset_filename}'")

# --- 3. Configure and Train the Fine-Tuning Model ---
MODEL_NAME = "distilgpt2"
OUTPUT_DIR = "./career-ai-roadmap-model"

# (The fine-tuning process remains the same as before)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

def load_dataset(file_path, tokenizer):
    return TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=256)

train_dataset = load_dataset(dataset_filename, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, overwrite_output_dir=True, num_train_epochs=15,
    per_device_train_batch_size=1, save_steps=50, save_total_limit=2, logging_steps=10,
)
trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=train_dataset)

print("\n--- Starting Model Fine-Tuning for Roadmap Generation ---")
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"--- Model Fine-Tuning Complete ---")
print(f"✅ Final roadmap model saved to '{OUTPUT_DIR}'")


# --- 4. Define the Roadmap and Course Content Generation Modules ---

def generate_roadmap(profile_text, experience_level):
    """Generates a career roadmap using the fine-tuned model."""
    loaded_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    loaded_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
    prompt = f"PROFILE: {profile_text}. LEVEL: {experience_level}. ROADMAP:"
    inputs = loaded_tokenizer(prompt, return_tensors="pt")
    outputs = loaded_model.generate(
        inputs.input_ids, max_length=300, num_return_sequences=1,
        pad_token_id=loaded_tokenizer.pad_token_id, eos_token_id=loaded_tokenizer.eos_token_id,
        do_sample=True, temperature=0.7, top_k=50
    )
    generated_text = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    roadmap = generated_text.split("ROADMAP:")[1].strip()
    return roadmap

def generate_course_content(profile_text, topic, api_key):
    """
    Generates detailed course content using the DeepSeek R1 model via a direct
    requests call to OpenRouter, as requested.
    """
    if not api_key:
        return "Error: OPENROUTER_API_KEY is not set. Cannot generate course content."

    prompt = f"""
    You are an expert curriculum designer. Your task is to generate a complete,
    in-depth lesson on the topic: "{topic}".

    The target audience is a student with the following profile: "{profile_text}".
    Tailor the complexity and examples to their background.

    The lesson should be structured as follows:
    1.  **Introduction**: Briefly explain what the topic is and why it's important.
    2.  **Core Concepts**: Break down the topic into 3-4 fundamental concepts. Explain each one clearly.
    3.  **Code Examples**: Provide clear, well-commented code examples where applicable.
    4.  **Practical Application**: Describe a real-world scenario where this knowledge is applied.
    5.  **Conclusion & Next Steps**: Summarize the key takeaways and suggest what to learn next.

    Generate the content in plain text format.
    """

    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json",
                # Optional headers for OpenRouter rankings
                "HTTP-Referer": "http://localhost",
                "X-Title": "Career AI Generator",
            },
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [
                    {"role": "system", "content": "You are an expert curriculum designer."},
                    {"role": "user", "content": prompt}
                ],
            })
        )

        response.raise_for_status() # Will raise an exception for bad status codes

        response_data = response.json()
        return response_data['choices'][0]['message']['content']

    except requests.exceptions.RequestException as e:
        return f"An error occurred with the API request: {e}"
    except (KeyError, IndexError) as e:
        return f"Error parsing the API response: {e}. Response: {response.text}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"


def create_voice_content(text, filename="course_audio.mp3"):
    """Converts text content to an MP3 audio file."""
    try:
        tts = gTTS(text=text, lang='en')
        tts.save(filename)
        print(f"✅ Voice content saved to '{filename}'")
        return filename
    except Exception as e:
        print(f"Could not create voice content: {e}")
        return None

# --- 5. Example Usage ---
print("\n--- Testing the Full Career AI System ---")

# IMPORTANT: Set your OpenRouter API key here
# You can get one from https://openrouter.ai/keys
OPENROUTER_API_KEY = "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"

new_profile = "Priya Singh, Computer Science, 2024, Python, Java, SQL"
experience_level = "Intermediate"

print(f"Profile: {new_profile}")
print(f"Requested Level: {experience_level}\n")

# Step 1: Generate the personalized roadmap
print("--- Generating Personalized Roadmap ---")
roadmap = generate_roadmap(new_profile, experience_level)
print(roadmap)

# Step 2: Choose a topic from the roadmap to generate course content for
# For this example, we'll pick "Docker & Containers" from the intermediate roadmap
course_topic = "Docker & Containers"
print(f"\n--- Generating Course Content for: '{course_topic}' ---")

# Step 3: Generate the detailed text-based course
course_text = generate_course_content(new_profile, course_topic, OPENROUTER_API_KEY)
print(course_text)

# Step 4: Generate the voice content for the course
print("\n--- Generating Voice Content ---")
create_voice_content(course_text)


✅ Roadmap dataset created at 'roadmap_data.jsonl'


/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(



--- Starting Model Fine-Tuning for Roadmap Generation ---


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.911600
20,1.956600
30,1.557700


--- Model Fine-Tuning Complete ---
✅ Final roadmap model saved to './career-ai-roadmap-model'

--- Testing the Full Career AI System ---
Profile: Priya Singh, Computer Science, 2024, Python, Java, SQL
Requested Level: Intermediate

--- Generating Personalized Roadmap ---
Model2D Printing. LEVEL: Intermediate.

--- Generating Course Content for: 'Docker & Containers' ---
Okay, Priya! Let's dive into a very powerful and widely-used technology in the modern software world: Docker and Containers.

Here is a lesson tailored for you:

---

# Lesson: Understanding Docker & Containers

Priya, with your background in Python and even other languages like Java and SQL, you've already worked with the complexities of managing dependencies and environments. Setting up a project on your machine means configuring libraries, specific versions, and sometimes even entire runtime environments. What if you could package your application *and* its environment neatly into a single unit that runs consistently

'course_audio.mp3'

In [ ]:
# main.py
# This script fine-tunes a model and then uses a powerful LLM to generate
# detailed course content, a certificate, and a LinkedIn post.

# --- 1. Installation ---
# pip install transformers torch datasets requests gtts
# 'requests' is used to interface with OpenRouter, and 'gtts' for text-to-speech.

import os
import json
import requests # Using the requests library to connect to OpenRouter
from gtts import gTTS
from datetime import datetime
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

# --- 2. Create an Expanded Sample Dataset for Roadmaps ---
# Added an "Advanced" level to make the fine-tuned model more robust.
roadmap_dataset_content = [
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Beginner. ROADMAP: ### Beginner Roadmap for Backend Developer\n\n**Month 1: Core Concepts**\n- **Courses:** Complete 'Introduction to Web Development' on Coursera. Study HTTP protocols and REST API principles.\n- **Project:** Build a simple command-line application that interacts with a public API (e.g., a weather app).\n\n**Month 2: Framework Fundamentals**\n- **Courses:** Learn the basics of a web framework like Django or FastAPI. Follow the official tutorials.\n- **Project:** Convert your command-line app into a basic web server with a few API endpoints.\n\n**Month 3: Database Essentials**\n- **Courses:** Take a course on SQL fundamentals and database design. Learn about ORMs (Object-Relational Mapping).\n- **Project:** Integrate a SQLite database into your web application to store and retrieve data."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Intermediate. ROADMAP: ### Intermediate Roadmap for Backend Developer\n\n**Month 1: Authentication & Security**\n- **Courses:** Study token-based authentication (JWT) and OAuth2. Learn about common security vulnerabilities (e.g., SQL injection, XSS).\n- **Project:** Implement user registration and login with secure password hashing and JWTs in your application.\n\n**Month 2: Docker & Containers**\n- **Courses:** Learn Docker fundamentals. Understand how to create Dockerfiles and manage containers with Docker Compose.\n- **Project:** Containerize your web application and its database into separate Docker containers.\n\n**Month 3: Testing & CI/CD**\n- **Courses:** Learn about unit testing and integration testing. Study the basics of CI/CD pipelines using GitHub Actions.\n- **Project:** Write unit tests for your API endpoints. Create a simple CI/CD pipeline that automatically runs tests when you push code to GitHub."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Advanced. ROADMAP: ### Advanced Roadmap for Backend Developer\n\n**Month 1: System Design & Scalability**\n- **Courses:** Study system design principles like load balancing, caching, and message queues (e.g., RabbitMQ, Kafka).\n- **Project:** Design a scalable architecture for a social media application, focusing on handling high traffic.\n\n**Month 2: Microservices & Orchestration**\n- **Courses:** Learn about microservice architecture. Study Kubernetes for container orchestration.\n- **Project:** Break down your monolithic application into several microservices and deploy them on a local Kubernetes cluster (Minikube).\n\n**Month 3: Performance Monitoring & Optimization**\n- **Courses:** Learn to use monitoring tools like Prometheus and Grafana. Study database indexing and query optimization.\n- **Project:** Set up a monitoring dashboard for your microservices. Identify and fix a performance bottleneck in your application."},
]

dataset_filename = "roadmap_data.jsonl"
with open(dataset_filename, 'w') as f:
    for entry in roadmap_dataset_content:
        f.write(json.dumps(entry) + '\n')

print(f"✅ Roadmap dataset created at '{dataset_filename}'")

# --- 3. Configure and Train the Fine-Tuning Model ---
MODEL_NAME = "distilgpt2"
OUTPUT_DIR = "./career-ai-roadmap-model"

# (The fine-tuning process remains the same as before)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

def load_dataset(file_path, tokenizer):
    return TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=256)

train_dataset = load_dataset(dataset_filename, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, overwrite_output_dir=True, num_train_epochs=15,
    per_device_train_batch_size=1, save_steps=50, save_total_limit=2, logging_steps=10,
)
trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=train_dataset)

print("\n--- Starting Model Fine-Tuning for Roadmap Generation ---")
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"--- Model Fine-Tuning Complete ---")
print(f"✅ Final roadmap model saved to '{OUTPUT_DIR}'")


# --- 4. Define the Roadmap, Course, Certificate, and Social Modules ---

def generate_roadmap(profile_text, experience_level):
    """Generates a career roadmap using the fine-tuned model."""
    loaded_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    loaded_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
    prompt = f"PROFILE: {profile_text}. LEVEL: {experience_level}. ROADMAP:"
    inputs = loaded_tokenizer(prompt, return_tensors="pt")
    outputs = loaded_model.generate(
        inputs.input_ids, max_length=300, num_return_sequences=1,
        pad_token_id=loaded_tokenizer.pad_token_id, eos_token_id=loaded_tokenizer.eos_token_id,
        do_sample=True, temperature=0.7, top_k=50
    )
    generated_text = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    roadmap = generated_text.split("ROADMAP:")[1].strip()
    return roadmap

def generate_course_content(profile_text, topic, api_key):
    """
    Generates detailed course content using the DeepSeek R1 model via a direct
    requests call to OpenRouter.
    """
    if not api_key:
        return "Error: OPENROUTER_API_KEY is not set. Cannot generate course content."

    prompt = f"""
    You are an expert curriculum designer. Your task is to generate a complete,
    in-depth lesson on the topic: "{topic}". The target audience is a student with
    the following profile: "{profile_text}". Tailor the complexity and examples
    to their background. The lesson should be structured with an Introduction,
    Core Concepts, Code Examples, Practical Application, and a Conclusion.
    Generate the content in plain text format.
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}", "Content-Type": "application/json",
                "HTTP-Referer": "http://localhost", "X-Title": "Career AI Generator",
            },
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [{"role": "user", "content": prompt}],
            })
        )
        response.raise_for_status()
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred: {e}"

def create_voice_content(text, filename="course_audio.mp3"):
    """Converts text content to an MP3 audio file."""
    try:
        tts = gTTS(text=text, lang='en')
        tts.save(filename)
        print(f"✅ Voice content saved to '{filename}'")
        return filename
    except Exception as e:
        print(f"Could not create voice content: {e}")
        return None

def generate_certificate(profile_name, course_topic):
    """Generates a simple text-based certificate of completion."""
    completion_date = datetime.now().strftime("%B %d, %Y")
    certificate = f"""
    ***************************************************
    * *
    * CERTIFICATE OF COMPLETION        *
    * *
    * This certificate is proudly presented to     *
    * *
    * {profile_name.center(20)}      *
    * *
    * For successfully completing the course on    *
    * *
    * "{course_topic.center(25)}"           *
    * *
    * Issued on: {completion_date.center(27)}      *
    * *
    * Career AI Platform             *
    * *
    ***************************************************
    """
    return certificate

def suggest_linkedin_post(profile_name, course_topic, api_key):
    """Generates a LinkedIn post suggestion using an LLM."""
    if not api_key:
        return "Error: API key not provided."

    prompt = f"""
    You are a LinkedIn career coach. Write an engaging and professional LinkedIn post for a user named '{profile_name}'
    who has just completed a course on '{course_topic}'. The post should be enthusiastic, mention the key skills learned,
    and include relevant hashtags.
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}", "Content-Type": "application/json",
            },
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [{"role": "user", "content": prompt}],
            })
        )
        response.raise_for_status()
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred while generating the LinkedIn post: {e}"


# --- 5. Example Usage ---
print("\n--- Testing the Full Career AI System ---")

OPENROUTER_API_KEY = "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"

# Extracting the name for the certificate and post
full_profile = "Priya Singh, Computer Science, 2024, Python, Java, SQL"
profile_name = full_profile.split(',')[0]
experience_level = "Intermediate"

print(f"Profile: {full_profile}")
print(f"Requested Level: {experience_level}\n")

# Step 1: Generate the personalized roadmap
print("--- Generating Personalized Roadmap ---")
roadmap = generate_roadmap(full_profile, experience_level)
print(roadmap)

# Step 2: Choose a topic and generate course content
course_topic = "Docker & Containers"
print(f"\n--- Generating Course Content for: '{course_topic}' ---")
course_text = generate_course_content(full_profile, course_topic, OPENROUTER_API_KEY)
print(course_text)

# Step 3: Generate the voice content for the course
print("\n--- Generating Voice Content ---")
create_voice_content(course_text)

# Step 4: Simulate course completion and generate certificate
print("\n--- Generating Certificate of Completion ---")
certificate = generate_certificate(profile_name, course_topic)
print(certificate)

# Step 5: Suggest a LinkedIn post for the user
print("\n--- Generating LinkedIn Post Suggestion ---")
linkedin_post = suggest_linkedin_post(profile_name, course_topic, OPENROUTER_API_KEY)
print(linkedin_post)


✅ Roadmap dataset created at 'roadmap_data.jsonl'


/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(



--- Starting Model Fine-Tuning for Roadmap Generation ---


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.911600
20,1.956600
30,1.557700


--- Model Fine-Tuning Complete ---
✅ Final roadmap model saved to './career-ai-roadmap-model'

--- Testing the Full Career AI System ---
Profile: Priya Singh, Computer Science, 2024, Python, Java, SQL
Requested Level: Intermediate

--- Generating Personalized Roadmap ---
Model2D Printing. LEVEL: Intermediate.

--- Generating Course Content for: 'Docker & Containers' ---
Okay, Priya! Let's dive into the world of Docker and Containers. Knowing you're a Computer Science student (especially with Python and Java experience) familiar with fundamentals like variables or database structures, I'll introduce the core concepts clearly and provide practical examples.

---

# Lesson: Understanding Docker & Containers

**For:** Priya Singh (Computer Science Student, PY/Java/SQL Background)
**Topic:** Docker & Containers

## 1. Introduction: Hello Consistency! (What is the Problem?)

Imagine you've built a Python script that uses a specific library. It works perfectly on your **local** machine. You g

In [ ]:
# main.py
# This script fine-tunes a model and then uses a powerful LLM to generate
# detailed course content, a certificate, a LinkedIn post, and a project plan.

# --- 1. Installation ---
# pip install transformers torch datasets requests gtts
# 'requests' is used to interface with OpenRouter, and 'gtts' for text-to-speech.

import os
import json
import requests # Using the requests library to connect to OpenRouter
from gtts import gTTS
from datetime import datetime
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

# --- 2. Create an Expanded Sample Dataset for Roadmaps ---
# Added an "Advanced" level to make the fine-tuned model more robust.
roadmap_dataset_content = [
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Beginner. ROADMAP: ### Beginner Roadmap for Backend Developer\n\n**Month 1: Core Concepts**\n- **Courses:** Complete 'Introduction to Web Development' on Coursera. Study HTTP protocols and REST API principles.\n- **Project:** Build a simple command-line application that interacts with a public API (e.g., a weather app).\n\n**Month 2: Framework Fundamentals**\n- **Courses:** Learn the basics of a web framework like Django or FastAPI. Follow the official tutorials.\n- **Project:** Convert your command-line app into a basic web server with a few API endpoints.\n\n**Month 3: Database Essentials**\n- **Courses:** Take a course on SQL fundamentals and database design. Learn about ORMs (Object-Relational Mapping).\n- **Project:** Integrate a SQLite database into your web application to store and retrieve data."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Intermediate. ROADMAP: ### Intermediate Roadmap for Backend Developer\n\n**Month 1: Authentication & Security**\n- **Courses:** Study token-based authentication (JWT) and OAuth2. Learn about common security vulnerabilities (e.g., SQL injection, XSS).\n- **Project:** Implement user registration and login with secure password hashing and JWTs in your application.\n\n**Month 2: Docker & Containers**\n- **Courses:** Learn Docker fundamentals. Understand how to create Dockerfiles and manage containers with Docker Compose.\n- **Project:** Containerize your web application and its database into separate Docker containers.\n\n**Month 3: Testing & CI/CD**\n- **Courses:** Learn about unit testing and integration testing. Study the basics of CI/CD pipelines using GitHub Actions.\n- **Project:** Write unit tests for your API endpoints. Create a simple CI/CD pipeline that automatically runs tests when you push code to GitHub."},
    {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Advanced. ROADMAP: ### Advanced Roadmap for Backend Developer\n\n**Month 1: System Design & Scalability**\n- **Courses:** Study system design principles like load balancing, caching, and message queues (e.g., RabbitMQ, Kafka).\n- **Project:** Design a scalable architecture for a social media application, focusing on handling high traffic.\n\n**Month 2: Microservices & Orchestration**\n- **Courses:** Learn about microservice architecture. Study Kubernetes for container orchestration.\n- **Project:** Break down your monolithic application into several microservices and deploy them on a local Kubernetes cluster (Minikube).\n\n**Month 3: Performance Monitoring & Optimization**\n- **Courses:** Learn to use monitoring tools like Prometheus and Grafana. Study database indexing and query optimization.\n- **Project:** Set up a monitoring dashboard for your microservices. Identify and fix a performance bottleneck in your application."},
]

dataset_filename = "roadmap_data.jsonl"
with open(dataset_filename, 'w') as f:
    for entry in roadmap_dataset_content:
        f.write(json.dumps(entry) + '\n')

print(f"✅ Roadmap dataset created at '{dataset_filename}'")

# --- 3. Configure and Train the Fine-Tuning Model ---
MODEL_NAME = "distilgpt2"
OUTPUT_DIR = "./career-ai-roadmap-model"

# (The fine-tuning process remains the same as before)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))

def load_dataset(file_path, tokenizer):
    return TextDataset(tokenizer=tokenizer, file_path=file_path, block_size=256)

train_dataset = load_dataset(dataset_filename, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR, overwrite_output_dir=True, num_train_epochs=15,
    per_device_train_batch_size=1, save_steps=50, save_total_limit=2, logging_steps=10,
)
trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=train_dataset)

print("\n--- Starting Model Fine-Tuning for Roadmap Generation ---")
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"--- Model Fine-Tuning Complete ---")
print(f"✅ Final roadmap model saved to '{OUTPUT_DIR}'")


# --- 4. Define the Roadmap, Course, Certificate, and Social Modules ---

def generate_roadmap(profile_text, experience_level):
    """Generates a career roadmap using the fine-tuned model."""
    loaded_tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
    loaded_model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)
    prompt = f"PROFILE: {profile_text}. LEVEL: {experience_level}. ROADMAP:"
    inputs = loaded_tokenizer(prompt, return_tensors="pt")
    outputs = loaded_model.generate(
        inputs.input_ids, max_length=300, num_return_sequences=1,
        pad_token_id=loaded_tokenizer.pad_token_id, eos_token_id=loaded_tokenizer.eos_token_id,
        do_sample=True, temperature=0.7, top_k=50
    )
    generated_text = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    roadmap = generated_text.split("ROADMAP:")[1].strip()
    return roadmap

def generate_course_content(profile_text, topic, api_key):
    """
    Generates detailed course content using the DeepSeek R1 model via a direct
    requests call to OpenRouter.
    """
    if not api_key:
        return "Error: OPENROUTER_API_KEY is not set. Cannot generate course content."

    prompt = f"""
    You are an expert curriculum designer. Your task is to generate a complete,
    in-depth lesson on the topic: "{topic}". The target audience is a student with
    the following profile: "{profile_text}". Tailor the complexity and examples
    to their background. The lesson should be structured with an Introduction,
    Core Concepts, Code Examples, Practical Application, and a Conclusion.
    Generate the content in plain text format.
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}", "Content-Type": "application/json",
                "HTTP-Referer": "http://localhost", "X-Title": "Career AI Generator",
            },
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [{"role": "user", "content": prompt}],
            })
        )
        response.raise_for_status()
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred: {e}"

def create_voice_content(text, filename="course_audio.mp3"):
    """Converts text content to an MP3 audio file."""
    try:
        tts = gTTS(text=text, lang='en')
        tts.save(filename)
        print(f"✅ Voice content saved to '{filename}'")
        return filename
    except Exception as e:
        print(f"Could not create voice content: {e}")
        return None

def generate_certificate(profile_name, course_topic):
    """Generates a simple text-based certificate of completion."""
    completion_date = datetime.now().strftime("%B %d, %Y")
    certificate = f"""
    ***************************************************
    * *
    * CERTIFICATE OF COMPLETION             *
    * *
    * This certificate is proudly presented to     *
    * *
    * {profile_name.center(20)}           *
    * *
    * For successfully completing the course on     *
    * *
    * "{course_topic.center(25)}"            *
    * *
    * Issued on: {completion_date.center(27)}       *
    * *
    * Career AI Platform                *
    * *
    ***************************************************
    """
    return certificate

def suggest_linkedin_post(profile_name, course_topic, api_key):
    """Generates a LinkedIn post suggestion using an LLM."""
    if not api_key:
        return "Error: API key not provided."

    prompt = f"""
    You are a LinkedIn career coach. Write an engaging and professional LinkedIn post for a user named '{profile_name}'
    who has just completed a course on '{course_topic}'. The post should be enthusiastic, mention the key skills learned,
    and include relevant hashtags.
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api_key}", "Content-Type": "application/json",
            },
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [{"role": "user", "content": prompt}],
            })
        )
        response.raise_for_status()
        response_data = response.json()
        return response_data['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred while generating the LinkedIn post: {e}"

def suggest_project(profile_text, roadmap_text, api_key):
    """Suggests a capstone project based on the user's profile and roadmap."""
    if not api_key:
        return "Error: API key not provided."

    prompt = f"""
    You are a senior technical mentor. Based on the user's profile and the learning roadmap they have,
    suggest a single, compelling capstone project that would allow them to showcase all the skills they've learned.

    USER PROFILE:
    {profile_text}

    LEARNING ROADMAP:
    {roadmap_text}

    Suggest a project with a clear title and a 2-3 sentence description of what it is and why it's a good fit.
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [{"role": "user", "content": prompt}],
            })
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred while suggesting a project: {e}"

def generate_project_plan(project_suggestion, api_key):
    """Generates a step-by-step plan to build the suggested project."""
    if not api_key:
        return "Error: API key not provided."

    prompt = f"""
    You are a project manager. A user has been given the following project suggestion.
    Your task is to create a clear, step-by-step plan to help them build it.

    PROJECT SUGGESTION:
    {project_suggestion}

    Create a plan with the following sections:
    1.  **Tech Stack**: List the recommended technologies (e.g., Python, Django, React, Docker).
    2.  **Core Features (MVP)**: List 3-5 essential features for the first version.
    3.  **Step-by-Step Build Guide**: Provide a 5-7 step guide from setting up the project to deploying it.
    4.  **Next Steps**: Suggest 2-3 advanced features to add after the MVP is complete.
    """
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({
                "model": "deepseek/deepseek-r1-0528-qwen3-8b:free",
                "messages": [{"role": "user", "content": prompt}],
            })
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        return f"An error occurred while generating the project plan: {e}"


# --- 5. Example Usage ---
print("\n--- Testing the Full Career AI System ---")

OPENROUTER_API_KEY = "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"

# Extracting the name for the certificate and post
full_profile = "Priya Singh, Computer Science, 2024, Python, Java, SQL"
profile_name = full_profile.split(',')[0]
experience_level = "Intermediate"

print(f"Profile: {full_profile}")
print(f"Requested Level: {experience_level}\n")

# Step 1: Generate the personalized roadmap
print("--- Generating Personalized Roadmap ---")
roadmap = generate_roadmap(full_profile, experience_level)
print(roadmap)

# Step 2: Choose a topic and generate course content
course_topic = "Docker & Containers"
print(f"\n--- Generating Course Content for: '{course_topic}' ---")
course_text = generate_course_content(full_profile, course_topic, OPENROUTER_API_KEY)
print(course_text)

# Step 3: Generate the voice content for the course
print("\n--- Generating Voice Content ---")
create_voice_content(course_text)

# Step 4: Simulate course completion and generate certificate
print("\n--- Generating Certificate of Completion ---")
certificate = generate_certificate(profile_name, course_topic)
print(certificate)

# Step 5: Suggest a LinkedIn post for the user
print("\n--- Generating LinkedIn Post Suggestion ---")
linkedin_post = suggest_linkedin_post(profile_name, course_topic, OPENROUTER_API_KEY)
print(linkedin_post)

# Step 6: Suggest a capstone project based on the roadmap
print("\n--- Generating Project Suggestion ---")
project_suggestion = suggest_project(full_profile, roadmap, OPENROUTER_API_KEY)
print(project_suggestion)

# Step 7: Generate a step-by-step plan for the project
print("\n--- Generating Project Build Plan ---")
project_plan = generate_project_plan(project_suggestion, OPENROUTER_API_KEY)
print(project_plan)


✅ Roadmap dataset created at 'roadmap_data.jsonl'


/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(



--- Starting Model Fine-Tuning for Roadmap Generation ---


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,2.911600
20,1.956600
30,1.557700


--- Model Fine-Tuning Complete ---
✅ Final roadmap model saved to './career-ai-roadmap-model'

--- Testing the Full Career AI System ---
Profile: Priya Singh, Computer Science, 2024, Python, Java, SQL
Requested Level: Intermediate

--- Generating Personalized Roadmap ---
Model2D Printing. LEVEL: Intermediate.

--- Generating Course Content for: 'Docker & Containers' ---
Okay, Priya! Let's dive into understanding Docker and how it revolutionizes software development. Having a background in Python, Java, and SQL is a great start – we'll apply these concepts directly.

---

**Lesson: Understanding Docker & Containers**

**Student:** Priya Singh
**Course:** Computer Science - 2024
**Background Knowledge:** Python, Java, SQL

**Lesson Duration:** Approx. 60-90 minutes

**Objective:** By the end of this lesson, Priya will be able to define containers, understand how Docker works, utilize basic Docker commands, and see the practical benefits of containerization in software development.

---

